# The building blocks of DL 2

> Simplest Neural Network: linear layer with activation function

- toc: false 
- badges: true
- comments: true
- categories: [medicalnist, mnist, basic]

In the [previous post](https://ml4med.github.io/blog/basic/sgd/2020/07/20/building-blocks-of-dl-sgd.html) I've explained what is the most important concept in neural networks - technique that allows us to incrementally find minimum of a function. This is called Gradient Descent algorithm!

In this post I will build on this concept and show you how to create a basic linear model to predict what is on a medical image!

### Download data

As in the [first post](https://ml4med.github.io/blog/medicalnist/mnist/basic/2020/06/19/fist_post.html) showing how to build medical image recognition with pure statistics, we need to download data first. For some basic description of how the data looks like, see that [first post](https://ml4med.github.io/blog/medicalnist/mnist/basic/2020/06/19/fist_post.html) first ;)

In [1]:
! git clone https://github.com/apolanco3225/Medical-MNIST-Classification.git
! mv Medical-MNIST-Classification/resized/ ./medical_mnist
! rm -rf Medical-MNIST-Classification

Cloning into 'Medical-MNIST-Classification'...
remote: Enumerating objects: 58532, done.
remote: Total 58532 (delta 0), reused 0 (delta 0), pack-reused 58532
Receiving objects: 100% (58532/58532), 77.86 MiB | 20.76 MiB/s, done.
Resolving deltas: 100% (506/506), done.
Checking connectivity... done.
Checking out files: 100% (58959/58959), done.


In [2]:
from pathlib import Path

PATH = Path("medical_mnist/")

We have much more powerful tools now, so we will deal with all 6 classes now, but first need to prepare data:
 - all data needs to be numerical
 - it needs to be in arrays
 - it needs to be labeled

In [3]:
classes = !ls {PATH}
classes

['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']

### Prepare data

In [4]:
images = {}
for cls in classes:
    images[cls] = !ls {PATH/cls}

In [5]:
from PIL import Image

In [6]:
# ToTensor converts images to tensorts
from torchvision.transforms import ToTensor

In [21]:
import torch

image_tensors = {}

for cls in classes:
    image_tensors[cls] = torch.stack(
        [
            ToTensor()(
                Image.open(path)
            ).view(-1, 64 * 64).squeeze().float()/255 
            
            for path in (PATH/cls).iterdir()]
    )

In [8]:
for cls in classes:
    class_shape = image_tensors[cls].shape
    print(f"{cls} has {class_shape[0]} images of a size {class_shape[1:]}")

AbdomenCT has 10000 images of a size torch.Size([4096])
BreastMRI has 8954 images of a size torch.Size([4096])
CXR has 10000 images of a size torch.Size([4096])
ChestCT has 10000 images of a size torch.Size([4096])
Hand has 10000 images of a size torch.Size([4096])
HeadCT has 10000 images of a size torch.Size([4096])


In [9]:
x_train = torch.cat([image_tensors[cls] for cls in classes], dim=0)
y_train = torch.cat([torch.tensor([index] * image_tensors[cls].shape[0]) for index, cls in enumerate(classes)])

shuffle the dataset. This is important as if we don't do this, images from the classes that we train first will be effectively not as "fresh" in the memmory of the network

In [10]:
permutations = torch.randperm(x_train.shape[0])

In [11]:
x_train = x_train[permutations]
y_train = y_train[permutations]

create validation set that is 20% of the training set - this is important to asses the performance of the model. Validation set doesn't take part in training, so model is not biased towards those images (it cannot remember those exact images from training). This is essential to see how well model generalizes on examples it has not seen.

In [12]:
valid_pct = 0.2
valid_index = int(x_train.shape[0] * valid_pct)
valid_index

11790

In [13]:
# we take out first 20% of examples from the training set
x_valid = x_train[:valid_index]
y_valid = y_train[:valid_index]

x_train = x_train[valid_index:]
y_train = y_train[valid_index:]

In [14]:
x_train.shape, y_train.shape, x_valid.shape, y_valid.shape

(torch.Size([47164, 4096]),
 torch.Size([47164]),
 torch.Size([11790, 4096]),
 torch.Size([11790]))